# 📝 Compresión de video MP4 usando H.264 en Python
Este cuaderno permite:
- Seleccionar un archivo `.mp4`
- Verificar su formato y duración (máximo 30 segundos)
- Mostrar sus metadatos
- Comprimirlo usando el códec H.264
- Medir el tiempo de compresión
- Comparar tamaño original y comprimido

## Requisitos
- Python 3.x
- FFmpeg instalado y agregado a las variables de entorno (PATH)

## 📦 Instalación previa de librerías

In [1]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 🔧 Configuración de parámetros
Aquí puedes ajustar los parámetros de compresión.

In [2]:
# Selección de perfil: 'ultra', 'equilibrado', 'rapido'
PERFIL_COMPRESION = 'equilibrado'

def configurar_parametros(perfil):
    if perfil == 'ultra':
        return {'preset': 'veryslow', 'crf': '30'}  # Mucha compresión, más lento
    elif perfil == 'rapido':
        return {'preset': 'ultrafast', 'crf': '23'}  # Menos compresión, muy rápido
    else:
        return {'preset': 'medium', 'crf': '26'}     # Balanceado

# Aplicar configuración
parametros = configurar_parametros(PERFIL_COMPRESION)
FFMPEG_PRESET = parametros['preset']
FFMPEG_CRF = parametros['crf']

# Duración máxima permitida (segundos)
MAX_DURACION = 30.0

## 📂 Seleccionar archivo MP4

In [3]:
import tkinter as tk
from tkinter import filedialog

def seleccionar_archivo_mp4():
    """
    Abre diálogo para seleccionar un archivo y devuelve la ruta.
    """
    root = tk.Tk()
    root.withdraw()
    tipos = [("Archivos MP4", "*.mp4"), ("Todos los archivos", "*")]
    ruta = filedialog.askopenfilename(title="Seleccione un archivo MP4", filetypes=tipos)
    return ruta

## 📑 Obtener metadatos con ffprobe

In [4]:
import subprocess, json, sys, os, mimetypes

def obtener_metadatos_ffprobe(ruta):
    cmd = [
        'ffprobe', '-v', 'error',
        '-print_format', 'json',
        '-show_format',
        '-show_streams',
        ruta
    ]
    try:
        resultado = subprocess.run(cmd, capture_output=True, text=True, check=True)
    except FileNotFoundError:
        print("Error: ffprobe no encontrado. Asegúrate de tener FFmpeg instalado y ffprobe en tu PATH.")
        sys.exit(1)
    except subprocess.CalledProcessError as e:
        print(f"Error al ejecutar ffprobe: {e.stderr}")
        sys.exit(1)

    info = json.loads(resultado.stdout)
    fmt = info.get('format', {})
    return {
        'format_name': fmt.get('format_name', 'desconocido'),
        'duration': float(fmt.get('duration', 0.0)),
        'size': int(fmt.get('size', 0))
    }

## 🎥 Función para comprimir video a H.264

In [5]:
import sys, subprocess

def comprimir_h264(input_path, output_path):
    cmd = [
        'ffmpeg', '-y',
        '-i', input_path,
        '-c:v', 'libx264',
        '-preset', FFMPEG_PRESET,
        '-crf', FFMPEG_CRF,
        '-c:a', 'copy',
        output_path
    ]
    try:
        subprocess.run(cmd, check=True, capture_output=True, text=True)
    except FileNotFoundError:
        print("Error: ffmpeg no encontrado. Asegúrate de tener FFmpeg instalado y ffmpeg en tu PATH.")
        sys.exit(1)
    except subprocess.CalledProcessError as e:
        print(f"Error al comprimir video: {e.stderr}")
        sys.exit(1)

## 🚀 Ejecutar todo el proceso

In [ ]:
import time, os, mimetypes

def main():
    archivo = seleccionar_archivo_mp4()
    if not archivo:
        print("No se seleccionó ningún archivo.")
        return

    base, ext = os.path.splitext(archivo)
    if ext.lower() != '.mp4':
        print("Formato incorrecto: debe ser un archivo .mp4.")
        return

    datos_orig = obtener_metadatos_ffprobe(archivo)
    dur = datos_orig['duration']
    if dur > MAX_DURACION:
        print(f"Duración: {dur:.2f}s. Excede el límite de {MAX_DURACION:.0f}s.")
        return

    size_orig_mb = datos_orig['size'] / (1024 * 1024)
    mime, _ = mimetypes.guess_type(archivo)

    print("\n=== Metadatos antes de comprimir ===")
    print(f"Duración      : {dur:.2f} segundos")
    print(f"Tamaño        : {size_orig_mb:.2f} MB")
    print(f"Formato       : {datos_orig['format_name']}")
    print(f"Tipo MIME     : {mime or 'desconocido'}")

    ruta_salida = f"{base}_compressed.mp4"

    print("\nComprimiendo video con H.264... Esto puede tardar unos segundos.")
    inicio = time.time()
    comprimir_h264(archivo, ruta_salida)
    fin = time.time()
    tiempo_comp = fin - inicio

    datos_comp = obtener_metadatos_ffprobe(ruta_salida)
    size_comp_mb = datos_comp['size'] / (1024 * 1024)
    reduccion = size_orig_mb - size_comp_mb
    porcentaje = (reduccion / size_orig_mb * 100) if size_orig_mb > 0 else 0

    print("\n=== Resultado de la compresión ===")
    print(f"Archivo comprimido: {ruta_salida}")
    print(f"Tamaño original   : {size_orig_mb:.2f} MB")
    print(f"Tamaño comprimido : {size_comp_mb:.2f} MB")
    print(f"Reducción         : {reduccion:.2f} MB ({porcentaje:.1f}%)")
    print(f"Tiempo compresión : {tiempo_comp:.2f} segundos")

main()